In [1]:
import numpy as np
import awkward as ak
import pickle
import pandas as pd
from sys import getsizeof

from PionModel import *

# Most of the steps have been commented out to avoid accidentally overwriting existing data
# Also to avoid running out of memory (an issue with the basic interactive given with SSH)
# More at 11

In [2]:
# this one does not have the AHCAL data.
# filePath = "/home/rusack/shared/pickles/HGCAL_TestBeam/pkl_files/DataHGCALTB_fixwt_woAHCAL_4feat/"
# This one does not have the beamEn file so I cant separate the data correctly
# filePath = "/home/rusack/shared/pickles/HGCAL_TestBeam/pkl_files/HGCAL_TB_Data_pkl/"
filePath = "/home/rusack/shared/pickles/HGCAL_TestBeam/pkl_files/pkl_updat_relwt_ahcaltrim_TB/"

# recHitEn = pickle.load(open(filePath + "recHitEn" + ".pickle", "rb"))
Hit_Z = pickle.load(open(filePath + "Hit_Z" + ".pickle", "rb"))
beamEn = pickle.load(open(filePath + "beamEn" + ".pickle", "rb"))




In [ ]:
# Get all unique depth positions
# uniqueZ = np.unique(ak.flatten(Hit_Z))
# with open(f"/home/rusack/vadna042/DNN_HGCAL/PionData/DepthPositions50.pickle", "wb") as a:
#     pickle.dump(uniqueZ, a)
#     del a

In [5]:
#split the events Hit_Z and recHitEn based on the beam Energy. 
#The Test beam energies are 20, 50, 80, 100, 120, 200, 250, 300 GeV


# beamEnUnique = np.unique(beamEn)
# startSplit = 0
# i = 0
# x = beamEn[i]

# while i < len(beamEn):
    
#     tempX = beamEn[i]
#     if tempX != x:
#         val = int(x)
#         with open(f"/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{val}/recHitEn.pickle", "wb") as a:
#             pickle.dump(recHitEn[startSplit:i], a)
#         with open(f"/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{val}/Hit_Z.pickle", "wb") as a2:
#             pickle.dump(Hit_Z[startSplit:i], a2)
#         del a, a2
#         x = tempX
#         startSplit = i
#     i += 1
# val = int(x)
# with open(f"/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{val}/recHitEn.pickle", "wb") as a:
#     pickle.dump(recHitEn[startSplit:i], a)
# with open(f"/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{val}/Hit_Z.pickle", "wb") as a2:
#     pickle.dump(Hit_Z[startSplit:i], a2)
# del a, a2

In [3]:
# Load in the Hit_Z and recHitEn for each recently separated TB file.
# Need to sum all recHitEns per layer per event per testBeam
# Any layer without any hits will be set to 0. 
# 
testBeams = [20,50,80,100,120,200,250,300]

# The unique depth positions
with open("/home/rusack/vadna042/DNN_HGCAL/PionData/DepthPositions50.pickle", "rb") as a:
    depthZ = pickle.load(a)
    del a

#Load in the pion event energy and layer files to calculate the summed layer values as input for the network.
# Files are separated based on test beam energy. 

for x in testBeams:
    with open(f'/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{x}/Hit_Z.pickle', 'rb') as a1:
        # layers.append(pickle.load(a1))
        layers = pickle.load(a1)
        del a1
    with open(f'/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{x}/recHitEn.pickle', 'rb') as a2:
        # energy.append(pickle.load(a2))
        energy = pickle.load(a2)
        del a2
    
    temp = np.array([ ak.sum(energy[layers==lz], axis=1) for lz in depthZ ])
    
    with open(f"/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{x}/enSummed.pickle", "wb") as a:
        pickle.dump(temp, a)
        del a
    
    
    

In [7]:
# Create the files for completely summed energy events. 
# Just total all the rechit energy per event per test beam energy.

testBeams = [20,50,80,100,120,200,250,300]

for x in testBeams:
    with open(f'/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{x}/recHitEn.pickle', 'rb') as a2:
        # energy.append(pickle.load(a2))
        energy = pickle.load(a2)
        del a2
    
    temp = np.array([ak.sum(energy, axis=1)]).T
    
    with open(f"/home/rusack/vadna042/DNN_HGCAL/PionData/TB_{x}/totalSummed.pickle", "wb") as a:
        pickle.dump(temp, a)
        del a
    